In [26]:
import torch
from torch.autograd import Variable
from torchvision import models

from __future__ import print_function, division
import os
import torch
from torch.optim.optimizer import Optimizer
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io
import torch.nn as nn
import torch.nn.functional as F

from skimage.color import rgb2gray

from torch.utils.tensorboard import SummaryWriter

import PIL
from PIL import Image

from keras.datasets import mnist

from google.colab import output

In [27]:
!nvidia-smi

Wed Mar 10 14:50:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |   3594MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [28]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter() 

new_number_of_classes = 10

# creating pretrained model:
pretrained_vgg16_model = models.vgg16(pretrained=True)

pretrained_vgg16_model.classifier[6] = nn.Linear(4096, new_number_of_classes)
pretrained_vgg16_model.classifier.add_module("sigmoid", nn.Sigmoid())
pretrained_vgg16_model.cuda()
pretrained_vgg16_model.eval()


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [29]:
from torchvision import datasets
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import numpy as np
from skimage import io
from skimage.transform import resize
import glob

In [30]:
def read_img(file_name):
    img = io.imread(file_name)
    img = [resize(img, (224, 224))]
    img = torch.tensor(img)
    img = img.permute(0, 3, 1, 2)
    return img.float()

In [31]:
class CustomDataset(Dataset):
    def __init__(self, images_dir, batch_size=4, transforms=None):
        self.images_dir = images_dir
        self.transforms = transforms

        self.masks_files = glob.glob(self.images_dir + "/masks/*.jpg")
        self.no_masks_files = glob.glob(self.images_dir + "/no_mask/*.jpg")
        self.no_masks_N = len(self.no_masks_files)
        self.masks_N = len(self.masks_files)

        self.labels = np.array([np.repeat(1, self.masks_N), np.repeat(0, self.no_masks_N)]).reshape(-1)
        self.files = self.masks_files + self.no_masks_files


         
    def __len__(self):
        return (len(self.labels))
    
    def __getitem__(self, i):
        file = self.files[i]
        img = read_img(file)
        
        if self.transforms:
            img = self.transforms(img)
            
        return (img, self.labels[i])

 
#train_dataset = CustomDataset("./masks_data")

(train_X, train_y), (test_X, test_y) = mnist.load_data()

class MNISTDataset(Dataset):
  def __init__(self, root):
    self.root = root
    self.data = { "train_X": train_X, "train_y": train_y, "test_X": test_X, "test_y": test_y }

  def __len__(self):
    if self.root == 'test':
      return len(self.data["test_X"])
    else:
      return len(self.data["train_X"])

  def __getitem__(self, idx):
    if self.root == 'test':
      img = [resize(test_X[idx], (224, 224))]
      #print(img)
      img = torch.tensor(img*3)
      img = img.unsqueeze(0)
      img = img.permute(0, 1, 2, 3)

      return (img.float(), test_y[idx])
    else:
      img = [resize(train_X[idx], (224, 224))]
      #print(img)
      img = torch.tensor(img*3)
      img = img.unsqueeze(0)
      img = img.permute(0, 1, 2, 3)

      return (img.float(), train_y[idx])

train_dataset = MNISTDataset('test')
test_dataset = MNISTDataset('train')


In [32]:
#for i in range(len(train_dataset)):
#    sample = train_dataset[i]
#    print(sample[0].shape, sample[1])

In [33]:
def batch_to_one_hot(id, batch_size, nb_classes = 2):
    y_onehot = torch.FloatTensor(batch_size, nb_classes)
    y_onehot.zero_()
    y_onehot.scatter_(1, id.view(-1, 1), 1)
    return y_onehot.long()

In [34]:
labels = torch.tensor(np.array([1, 0, 0, 1]))
batch_to_one_hot(labels, 4, 2)

tensor([[0, 1],
        [1, 0],
        [1, 0],
        [0, 1]])

In [35]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss().cuda() 
optimizer = optim.SGD(pretrained_vgg16_model.parameters(), lr=0.001, momentum=0.9)

In [36]:
# number of epochs to train the model
n_epochs = 3

for epoch in range(1, n_epochs+1):
    # monitor training loss
    train_loss = 0.0
    validation_loss = 0.0
    
    ###################
    # train the model #
    ###################
    for i in range(len(train_dataset)):
        images, labels = train_dataset[i]
        
        # _ stands in for labels, here
        #images, labels = data
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        
        outputs = pretrained_vgg16_model(images.cuda())
        # calculate the loss
        labels = torch.tensor([labels], dtype=torch.long)

        loss = criterion(outputs, labels.cuda())

        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*images.size(-1)
        
        print("Epoch: {} Idx: {}/{} Loss: {}".format(epoch, i, len(train_dataset), loss))
        

    # print avg training statistics 
    train_loss = train_loss / len(train_dataset)
    print('Epoch: {} \tTraining Loss: {:.10f}'.format(
        epoch, 
        train_loss 
        ))

Streaming output truncated to the last 5000 lines.
Epoch: 2 Idx: 817/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 818/10000 Loss: 1.5979795455932617
Epoch: 2 Idx: 819/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 820/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 821/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 822/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 823/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 824/10000 Loss: 1.5979795455932617
Epoch: 2 Idx: 825/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 826/10000 Loss: 1.5979795455932617
Epoch: 2 Idx: 827/10000 Loss: 1.5979795455932617
Epoch: 2 Idx: 828/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 829/10000 Loss: 1.5979795455932617
Epoch: 2 Idx: 830/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 831/10000 Loss: 1.5979795455932617
Epoch: 2 Idx: 832/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 833/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 834/10000 Loss: 2.5979795455932617
Epoch: 2 Idx: 835/10000 Loss: 1.5979795455932617
Epoch: 2 Idx: 836/

KeyboardInterrupt: ignored

In [ ]:
increaseValues = np.zeros(len(train_dataset))**2
dataIdx = np.zeros(len(train_dataset))**2

calculatedIdx = 0
prevPercent = 0
for i in range(len(train_dataset)):
      images, labels = train_dataset[i]

      outputs = pretrained_vgg16_model(images.cuda())

      collumnIdx = 0
      for collumnIdx in range(len(train_dataset)):
          if(outputs[0].any() == labels):
            epsilon = 1
            increase = 0
            dataIdx[collumnIdx] += 1

            while(predicted == real_answer and increase < 100):
              tempData = dat[idx].flatten().float()
              increase += epsilon
              tempData[0] += increase

              outputs = pretrained_vgg16_model(torch.Tensor(tempData))

              predicted = get_answer(outputs)
              real_answer = labels[idx].item()

            increaseValues[collumnIdx] += increase

      calculatedIdx += 1
      if calculatedIdx*100/len(train_dataset) != prevPercent:
        prevPercent = calculatedIdx*100/len(train_dataset)
        print("{}%".format(prevPercent))

for i in range(len(train_dataset)):
  if dataIdx[i] == 0:
    dataIdx[i] = 1

  print("{}th collumn maximum epsilon: {}".format(i, increaseValues[i]/dataIdx[i]))

s = 0
for i in range(len(train_dataset)):
  s += increaseValues[i]/dataIdx[i]

print("Model maximum epsilon: {}".format(s/len(train_dataset)))

In [63]:
increaseValues = np.zeros(len(train_dataset))
dataIdx = np.zeros(len(train_dataset))

calculatedIdx = 0
prevPercent = 0

calculatedIdx = 0
prevPercent = 0
for i in range(len(train_dataset)):
  images, labels = train_dataset[i]

  outputs = pretrained_vgg16_model(images.cuda())

  if(outputs[0][labels-1] == 1):
    epsilon = 0.1
    increase = 0
    dataIdx[i] += 1

    while(outputs[0][labels-1] == 1 and increase < 100):
      tempData = images
      increase += epsilon
      tempData[0] = tempData[0] + increase

      outputs = pretrained_vgg16_model(tempData.cuda())

    increaseValues[i] += increase
    calculatedIdx += 1
 
  currPerc = i*100/len(train_dataset)
  if currPerc != prevPercent:
    prevPercent = currPerc
    print("{}%".format(currPerc))

for i in range(len(train_dataset)):
  if dataIdx[i] == 0:
    dataIdx[i] = 1

  print("{}th collumn maximum epsilon: {}".format(i, increaseValues[i]/dataIdx[i]))

s = 0
for i in range(len(train_dataset)):
  s += increaseValues[i]/dataIdx[i]

print("Model maximum epsilon: {}".format(s/len(train_dataset)))

Streaming output truncated to the last 5000 lines.
5001th collumn maximum epsilon: 0.0
5002th collumn maximum epsilon: 0.0
5003th collumn maximum epsilon: 0.0
5004th collumn maximum epsilon: 0.0
5005th collumn maximum epsilon: 0.0
5006th collumn maximum epsilon: 0.0
5007th collumn maximum epsilon: 0.0
5008th collumn maximum epsilon: 0.0
5009th collumn maximum epsilon: 0.0
5010th collumn maximum epsilon: 0.0
5011th collumn maximum epsilon: 0.0
5012th collumn maximum epsilon: 0.0
5013th collumn maximum epsilon: 0.0
5014th collumn maximum epsilon: 0.0
5015th collumn maximum epsilon: 0.0
5016th collumn maximum epsilon: 0.0
5017th collumn maximum epsilon: 0.0
5018th collumn maximum epsilon: 0.8999999999999999
5019th collumn maximum epsilon: 0.0
5020th collumn maximum epsilon: 0.8999999999999999
5021th collumn maximum epsilon: 0.8999999999999999
5022th collumn maximum epsilon: 0.0
5023th collumn maximum epsilon: 0.0
5024th collumn maximum epsilon: 0.8999999999999999
5025th collumn maximum ep